Overall intended workflow for Abidjan flood detection. Based on process established by Huang et al. (2015)
    - Import DG image downloaded from GBDX platform
    - Convert geotiff files to numpy array via rasterio
    - Scale to surface reflectance values (divide by 10,000)
    - Define individual bands for later calculations
    - Calculate indices (NDVI, multiple versions of NDWI) and dark parameter
    - Threshold and plot based on Cloud to Street algorithm
    - Calculate MSI 
    - Threshold and plot NDVI, NDWI and MSI
    - Compare output to Cloud-to-Street results
    - Segment final image 
    - Identify characteristics of objects (area, shape, density, length:width ratio, homogeneity)

**References**
Huang, X., Xie, C., Fang, X., Zhang, L. Combining Pixel-and Object-Based Machine Learning for Identification of Water-Body Types from Urban High-Resolution Remote-Sensing Imagery. IEEE J. Sel. Top. Appl. Earth Obs. Remote Sens. 2015, 8, 2097–2110.

Cloud to Street. Urban Flood Mapping Using Very-High Resolution Satellite Imagery. Available at: https://abidjan.cloudtostreet.info/info

This notebook calculates NDWI optimal and MSI per Xie et al. (2016) and subtracts MSI from NDWI, resulting in a final water mask. 

Cloud, MSI and NDWI images are morphologically opened to get rid of small one off pixels.
Opening done via scikit image opening package

In [ ]:
%matplotlib inline

In [ ]:
# Import necessary packages
import os
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from glob import glob
from skimage.morphology import erosion, dilation, opening, closing, black_tophat, white_tophat
from skimage import draw
from skimage.morphology import disk, diamond
import rasterio as rio
from rasterio.mask import mask
import earthpy as et
import earthpy.spatial as es

In [ ]:
# Set working directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics','spring-2019', 'flood-detection'))
os.environ["PROJ_LIB"] = r"C:\Users\katy\Anaconda3\envs\earth-analytics-python\Library\share"

### Update band_path with directory for imagery of interest

In [ ]:
aoi = "bietry_ps"
band_path = os.path.join("dg-data", aoi +"\\")
all_dg_bands = glob(band_path+"*band_*")
all_dg_bands.sort()
outputs_path = os.path.join(band_path, "outputs\\")

In [ ]:
# Create raster stack of all bands
all_bands_path = os.path.join(outputs_path, aoi + "_all_bands.tif")
all_bands, all_bands_meta = es.stack(all_dg_bands,
                                             all_bands_path)

In [ ]:
# Open raster stack
with rio.open(all_bands_path) as src:
    all_bands_stack = src.read()
    all_bands_stack_meta = src.profile
    files_out_meta = src.profile
    masks_out_meta = src.profile

In [ ]:
# # Update files out meta for writing out single files
# files_out_meta['count']=1
# # Update masks_out_meta for writing single integer type files
# masks_out_meta['dtype'] = 'int32'
# masks_out_meta['count'] = 1

In [ ]:
# Scale raster stack to surface reflectance values (should be between 0 and 1)
all_bands_scaled = all_bands_stack/10000

In [ ]:
# Commented out due to memory issues
# # Plot histogram to explore scaled value distribution 
# fig,ax = plt.subplots()
# plt.hist(all_bands_scaled.ravel(), bins='auto');

In [ ]:
# Check for max scaled values 
# print(all_bands_scaled[7].max(), all_bands_stack[7].max())

In [ ]:
# Commented out due to memory issues
#Plot one band to take quick look at image
# fig,ax = plt.subplots()
# plt.imshow(all_bands_scaled[0])

In [ ]:
# Read in scaled bands
coastal_scaled = all_bands_scaled[0]
blue_scaled = all_bands_scaled[1]
green_scaled = all_bands_scaled[2]
yellow_scaled = all_bands_scaled[3]
red_scaled = all_bands_scaled[4]
rededge_scaled = all_bands_scaled[5]
nearir1_scaled = all_bands_scaled[6]
nearir2_scaled = all_bands_scaled[7]

In [ ]:
# Calculate cloud mask layer based on Cloud to Street thresholds
cloud = ((coastal_scaled >= 0.25) & (rededge_scaled >= 0.25)).astype(int)

In [ ]:
# Trial with opening to get rid of areas identified in cloud mask which are not actually cloud
selem = disk(15)
cloud_opened = opening(cloud, selem)

In [ ]:
#Plot opened cloud mask
fig, ax = plt.subplots(figsize=(12, 8))
im = ax.imshow(cloud_opened,
               cmap=plt.cm.get_cmap('tab20b', 2))
cbar = fig.colorbar(im)
cbar.set_ticks((0.25, .75))
cbar.ax.set_yticklabels(["Clear Pixels", "Cloud / Shadow Pixels"])
ax.set_title("Cloud Mask | Light Purple Pixels will be Masked")
ax.set_axis_off()
plt.show()

In [ ]:
# Apply cloud mask to all_bands_scaled stack
cloud_mask = np.broadcast_to(cloud_opened == 1, all_bands_scaled.shape)
all_bands_cl_free = ma.masked_array(all_bands_scaled,
                                      mask=cloud_mask)

In [ ]:
# Define cloud masked scaled bands
coastal_cl_free = all_bands_cl_free[0]
blue_cl_free = all_bands_cl_free[1]
green_cl_free = all_bands_cl_free[2]
yellow_cl_free = all_bands_cl_free[3]
red_cl_free = all_bands_cl_free[4]
rededge_cl_free = all_bands_cl_free[5]
nearir1_cl_free = all_bands_cl_free[6]
nearir2_cl_free = all_bands_cl_free[7]

## Calculate MSI 

In [ ]:
# Function for calculating the linear structuring element
def selemline(length, theta, dtype=np.uint8):
    """Line structuring element"""
    theta_d = theta * np.pi / 180
    X = int(round((length-1)/2. * np.cos(theta_d)))
    Y = int(-round((length-1)/2. * np.sin(theta_d)))
    C, R, V = draw.line_aa(-X, -Y, X, Y)
    M = 2*max(abs(R)) + 1
    N = 2*max(abs(C)) + 1
    selem = np.zeros((M, N)).astype(dtype)
    selem[R + max(abs(R)), C + max(abs(C))] = 1
    return selem

In [ ]:
# Calculate brightness for cloud masked stack
brightness2=np.nanmax(all_bands_cl_free, axis=0)
# brightness = all_bands_cl_free.max(axis=0)
print(np.nanmax(brightness2), np.nanmin(brightness2))

In [ ]:
# Cap brightness values at a max of 1. Replace all values greater than 1 with a value of 1
brightness_cap = np.copy(brightness2)
# np.place(brightness_cap, brightness_cap > 1, 1)
brightness_cap=np.ma.where(brightness2>1,1,brightness2)
print(brightness_cap.max(), brightness_cap.min())

In [ ]:
# Plot brightness_cap image
fig, ax = plt.subplots(figsize=(20,10))
brightness_plot = ax.imshow(brightness_cap, cmap='RdBu',
                vmin=0, vmax=1)
fig.colorbar(brightness_plot, fraction=.05)
ax.set(title="Brightness, June 17, 2016\n"+aoi+"\n")
ax.set_axis_off()
plt.show()

In [ ]:
# Plot histogram to explore distribution of brightness_cap values
fig,ax = plt.subplots()
plt.hist(brightness_cap.ravel());

In [ ]:
# Check if you have initialized how you think you meant to
# Initialize inputs for MSI calculation 
selem = selemline(0,0)
b_tophat_array_sum = black_tophat(brightness2, selem)

In [ ]:
# Loop and sum black tophat morphological profiles for MSI calculation
for i in range(2,57,5):
    for x in range(0, 4):
        selem = selemline(i,45*x)
        b_tophat = black_tophat(brightness2,selem)
        b_tophat_array_sum = b_tophat_array_sum.__add__(b_tophat)

In [ ]:
msi = b_tophat_array_sum/(4*11)
print(msi.max())

In [ ]:
# Plot histogram to explore distribution of MSI values
fig,ax = plt.subplots()
plt.hist(msi.ravel());

In [ ]:
# Plot MSI
fig, ax = plt.subplots(figsize=(20,10))
msi_plot = ax.imshow(msi, cmap='PiYG',
                vmin=0, vmax=0.20)
fig.colorbar(msi_plot, fraction=0.05)
ax.set(title="MSI, June 17, 2016\n"+aoi+"\n")
ax.set_axis_off()
plt.show()

In [ ]:
# Develop shadow mask for MSI > 0.12 and apply morphological opening
shadow = (msi>=0.12).astype(int)
selem_shadow = disk(4)
shadow_opened = opening(shadow, selem_shadow)

In [ ]:
# Plot shadow mask layer 
fig, ax = plt.subplots(figsize=(20,10))
test_plot = ax.imshow(shadow_opened, cmap='Oranges',
                vmin=0, vmax=1)
fig.colorbar(test_plot, fraction=.05)
ax.set(title="MSI Shadow Mask, June 17, 2016\n"+aoi+"\n")
ax.set_axis_off()
plt.show()

In [ ]:
# with rio.open(outputs_path+'msi_shadow_mask_'+aoi+'.tif', 'w', **masks_out_meta) as dst:
#     dst.write(shadow_opened, 1)

# Calculate "optimal" NDWI values

In [ ]:
# Calculate two "optimal" NDWIs from Xie et al. (2016) paper
ndwi_coastal = es.normalized_diff(b1=coastal_cl_free, b2 = nearir2_cl_free)
ndwi_green = es.normalized_diff(b1=green_cl_free, b2 = nearir2_cl_free)
# Subtract MSI from NDWI optimal
ndwi_coastal_msi = ndwi_coastal - msi
ndwi_green_msi = ndwi_green - msi

In [ ]:
# Compare plots for NDWI_Coastal and NDWI_Green 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

plt.suptitle('NDWI_Coastal and NDWI_Green n June 17, 2016\n'+aoi, fontsize=16)

ndwi_coastal_plot = ax1.imshow(ndwi_coastal, cmap = 'Blues', vmin = -1, vmax = 1)
ax1.set(title="NDWI_Coastal_NIR2\n")
ax1.set_axis_off()

ndwi_green_plot = ax2.imshow(ndwi_green, cmap = 'Blues', vmin = -1, vmax = 1)
ax2.set(title="NDWI_Green_NIR2\n")
ax2.set_axis_off()

fig.colorbar(ndwi_coastal_plot, fraction=.05)
plt.show()

In [ ]:
# Compare plots for NDWI_Coastal - MSI and NDWI_Green - MSI
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

plt.suptitle('NDWI_Coastal - MSI and NDWI_Green - MSI\n June 17, 2016\n'+aoi, fontsize=16)

ndwi_coastal_plot = ax1.imshow(ndwi_coastal_msi, cmap = 'Blues', vmin = -1, vmax = 1)
ax1.set(title="NDWI_Coastal - MSI\n")
ax1.set_axis_off()

ndwi_green_plot = ax2.imshow(ndwi_green_msi, cmap = 'Blues', vmin = -1, vmax = 1)
ax2.set(title="NDWI_Green - MSI\n")
ax2.set_axis_off()

fig.colorbar(ndwi_coastal_plot, fraction=.05)
plt.show()

In [ ]:
# Calculate mask for ndwi-msi 
ndwi_coastal_msi_mask = (ndwi_coastal_msi>0.15).astype(int)
ndwi_green_msi_mask = (ndwi_green_msi>0.15).astype(int)

In [ ]:
# Compare plots for NDWI_Coastal - MSI and NDWI_Green - MSI masks
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

plt.suptitle('NDWI_Coastal - MSI and NDWI_Green - MSI Flood Masks\n June 17, 2016\n'+aoi, fontsize=16)

ndwi_coastal_mask_plot = ax1.imshow(ndwi_coastal_msi_mask, cmap = 'Blues', vmin = 0, vmax = 1)
ax1.set(title="NDWI_Coastal - MSI Flood Mask\n")
ax1.set_axis_off()

ndwi_green_mask_plot = ax2.imshow(ndwi_green_msi_mask, cmap = 'Blues', vmin = 0, vmax = 1)
ax2.set(title="NDWI_Green - MSI Flood Mask\n")
ax2.set_axis_off()

fig.colorbar(ndwi_coastal_mask_plot, fraction=.05)
plt.show()